# Scraping common Japanese words with examples

The goal is to obtain a csv file with data scraped from https://iknow.jp/courses

In [1]:
#import libraries
#https://www.linkedin.com/pulse/scraping-therapists-python-selenium-beautifulsoup-scott-edenbaum
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import random
import numpy as np
import lxml
import pandas as pd
import re
import sys
from bs4 import BeautifulSoup, Comment

def scrape_japanese(url):
    driver = webdriver.Chrome("C:/Users/karol/drivers/chromedriver")  #select selenium web driver
    driver.get(url) #open the url in selenium
    jpVocabTable = [] #this list will be populated with scraped text
    soup = BeautifulSoup(driver.page_source,'html5lib') #grab the content with beautifulsoup for parsing
    main_table = soup.findAll("li",{"class":"item"})  # select the desired html node

    for row in main_table:
        rowList = []
        
        ######## find japanese words ###########################
        
        href = row.contents[0].contents[2].contents[0].find('a', href=True)
        word = row.find_all('a', href=True)
        jpWord= href.text.replace('\'', '')
        
        ######## find transliteration ###########################
        
        translit = row.contents[0].contents[2].contents[0].find('span', {"class":"transliteration"})
        if translit is not None:
            comments = translit.findAll(text=lambda text:isinstance(text, Comment))
            for comment in comments:
                comment.extract()
            transliteration = re.sub('\[|\]', '', translit.text)
        else: #there are cases where there is no transliteration and we want them empty
            transliteration = ""

        ######## find English translation ###########################

        trans = row.contents[0].contents[2].find('p', {"class":"response"})
        #if trans is not None:  #some returning nones
        translation = trans.text.replace('\'', '')

         ######## find example sentence 1 ###########################

        sen1 = row.contents[1].contents[0].contents[2].find('span').text

        ######## find example sentence 1 transliteration ###########################

        sen1transli = row.contents[1].contents[0].contents[2].find('p', {"class":"transliteration"}).text

        ######## find example sentence 1 translation ###########################

        sen1transla = row.contents[1].contents[0].contents[2].find('p', {"class":"translation"}).text

        ######## find example sentence 2 ###########################
        try:
            sen2 = row.contents[1].contents[1].contents[2].find('span').text
        except IndexError as e:
            print(e)

        ######## find example sentence 2 transliteration ###########################
        try:
            sen2transli = row.contents[1].contents[1].contents[2].find('p', {"class":"transliteration"}).text
        except IndexError as e:
            print(e)

        ######## find example sentence 2 translation ###########################
        try:
            sen2transla = row.contents[1].contents[1].contents[2].find('p', {"class":"translation"}).text
        except IndexError as e:
            print(e)


        rowList.extend([jpWord,transliteration, translation, sen1, sen1transli, sen1transla, sen2, sen2transli, sen2transla])
        jpVocabTable.append(rowList)
            
    driver.close()    
    print("Scraping Complete!")
    return(jpVocabTable)
        
        
url = "https://iknow.jp/courses/566921" 
japanese_list = scrape_japanese(url)

list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out o

In [2]:
japanese_list

[['行く',
  'いく',
  'go',
  '日曜日は図書館に行きます。',
  'にちようび は としょかん に いきます。',
  'I go to the library on Sundays.',
  '私は夏休みにプールに行った。',
  'わたし は なつやすみ に プール に いった。',
  'I went to the pool during summer vacation.'],
 ['見る',
  'みる',
  'see, look at',
  '私は絵を見るのが好きです。',
  'わたし は え を みる の が すき です。',
  'I like looking at pictures.',
  '仕事のあと、映画を見た。',
  'しごと の あと、 えいが を みた。',
  'I saw a movie after work.'],
 ['多い',
  'おおい',
  'a lot of, many',
  '京都にはお寺が多い。',
  'きょうと に は おてら が おおい。',
  'There are a lot of temples in Kyoto.',
  'この道は車が多い。',
  'この みち は くるま が おおい。',
  'There are many cars on this road.'],
 ['家',
  'うち',
  'home, household',
  '家に遊びにきてください。',
  'うち に あそび に きて ください。',
  'Please come over to my home.',
  '夏休みにお祖母ちゃんの家へ行きました。',
  'なつやすみ に おばあちゃん の うち へ いきました。',
  "I went to my grandma's house during summer vacation."],
 ['これ',
  '',
  'this, this one',
  'これをください。',
  'これ を ください。',
  "I'll have this please.",
  'これはあなたの鞄ですか。',
  'これ は あなた の かばん です か。',
  'Is this your bag?'],
 ['それ',
  '',


In [3]:
import pandas as pd
df = pd.DataFrame(japanese_list)
df.columns = ['word', 'kana', 'translation', 'sentence1', 'sen1kana', 'sen1trans','sentence2','sen2kana', 'sen2trans']
df.to_csv('japaneseWords_1000.csv', encoding='utf8', index = False) #don't add an extra line for indexing
df.shape

(100, 9)

In [4]:
random_row = df.sample(n=1)
list_row =random_row.values.tolist()
list_row

[['体',
  'からだ',
  'body, physique, physical condition',
  '私は体が丈夫だ。',
  'わたし は からだ が じょうぶ だ。',
  "I'm physically strong.",
  'タバコは体に悪い。',
  'タバコ は からだ に わるい。',
  'Cigarettes are bad for your health.']]